In [1]:
import pandas as pd
import numpy as np

In [2]:
!pip install -U pandas-profiling
!pip install --upgrade pythainlp
!pip install pyLDAvis
!pip install scipy
!pip install pyLDAvis==2.1.2

     |████████████████████████████████| 256kB 25.7MB/s 
     |████████████████████████████████| 645kB 40.9MB/s 
     |████████████████████████████████| 112kB 47.7MB/s 
     |████████████████████████████████| 1.1MB 33.1MB/s 
     |████████████████████████████████| 61kB 9.3MB/s 
     |████████████████████████████████| 3.1MB 39.5MB/s 
     |████████████████████████████████| 10.1MB 31.8MB/s 
     |████████████████████████████████| 81kB 11.6MB/s 
     |████████████████████████████████| 296kB 48.0MB/s 
  Created wheel for phik: filename=phik-0.11.2-cp37-none-any.whl size=1107437 sha256=9cc4b82401038fcc75f7066b92a64b305e0a3f38d2392355c1e3669a5142e061
  Stored in directory: /root/.cache/pip/wheels/c0/a3/b0/f27b1cfe32ea131a3715169132ff6d85653789e80e966c3bf6
  Created wheel for htmlmin: filename=htmlmin-0.1.12-cp37-none-any.whl size=27099 sha256=e5a12f325aaa2502a58d4df9a08bbc0e841ad2944b2bf3ff5a02a6674309baa7
  Stored in directory: /root/.cache/pip/wheels/43/07/ac/7c5a9d708d65247ac1f94066cf1db07

     |████████████████████████████████| 1.6MB 28.1MB/s 
  Created wheel for pyLDAvis: filename=pyLDAvis-2.1.2-py2.py3-none-any.whl size=97738 sha256=b00926d5888961fe603e76f6631d547c4e7945010c1f07ac83cc784ba821a80d
  Stored in directory: /root/.cache/pip/wheels/98/71/24/513a99e58bb6b8465bae4d2d5e9dba8f0bef8179e3051ac414
Successfully built pyLDAvis
  Found existing installation: pyLDAvis 3.3.1
    Uninstalling pyLDAvis-3.3.1:
      Successfully uninstalled pyLDAvis-3.3.1


In [3]:
import pythainlp
import pyLDAvis
import gensim
# import pyLDAvis.gensim_models as gensimvis
pyLDAvis.enable_notebook()
#import warnings
#warnings.filterwarnings("ignore", category=DeprecationWarning) 

/usr/local/lib/python3.7/dist-packages/past/types/oldstr.py:5: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Iterable
/usr/local/lib/python3.7/dist-packages/past/builtins/misc.py:4: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Mapping
/usr/local/lib/python3.7/dist-packages/scipy/fft/__init__.py:97: DeprecationWarning: The module numpy.dual is deprecated.  Instead of using dual, use the functions directly from numpy or scipy.
  from numpy.dual import register_func
/usr/local/lib/python3.7/dist-packages/scipy/sparse/sparsetools.py:21: DeprecationWarning: `scipy.sparse.sparsetools` is deprecated!
scipy.sparse.sparsetools is a private module for scipy.sparse, and should not be used.
  _deprecated()


In [4]:
import pandas as pd

In [5]:
df = pd.read_csv('CustomerReviews.csv')

In [6]:
# df.head()
print(df.head())

   Review ID  ... Rating
0          1  ...    5.0
1          2  ...    5.0
2          3  ...    4.0
3          4  ...    4.0
4          5  ...    5.0

[5 rows x 7 columns]


In [7]:
stopwords = list(pythainlp.corpus.thai_stopwords())
removed_words = ['😆','🤣',' ', '  ', '\n','\n\n', 'ร้าน', '(', ')' , '           ','–', '!', '!!','-','/','+','"','','%','\u200b','::', ':','+']
screening_words = stopwords + removed_words

def tokenize_with_space(sentence):
  merged = ''
  words = pythainlp.word_tokenize(str(sentence), engine='newmm')
  for word in words:
    if word not in screening_words:
      merged = merged + ',' + word
  return merged[1:]

In [8]:
df['Review_tokenized'] = df['Review'].apply(lambda x: tokenize_with_space(x))

In [9]:
#df.tail()
print(df.tail())

    Review ID  ...                                   Review_tokenized
16         17  ...  หลังจากที่,ลอง,สาขา,ยู,นม,อลล์,รอบ,สาขา,เดอะ,ม...
17         18  ...  บุฟเฟ่ต์,โปรโมชั่น,หัว,199,บาท,น้ำ,VAT,ทาน,ธรร...
18         19  ...  รู้จัก,union,mall,ลอง,กิน,จำได้,ขึ้นใจ,ชื่อ,ชื...
19         20  ...  อร่อย,คุ้มค่า,สมราคา,บุ,ฟเฟ่,หมู,ผัก,น้ำ,จบ,25...
20         21  ...  จ่าย,ราคา,199,บาท,จ่าย,เงินสด,ราคา,น้ำ,VAT,7,ส...

[5 rows x 8 columns]


In [10]:
documents = df['Review_tokenized'].to_list()
texts = [[text for text in doc.split(',')] for doc in documents]
dictionary = gensim.corpora.Dictionary(texts)

In [11]:
print(dictionary.token2id.keys())

dict_keys(['1', '4', '560', 'กิน', 'ข้างนอก', 'ความต้องการ', 'คอ', 'คุณภาพ', 'คุ้ม', 'จำกัด', 'ซุป', 'ดำ', 'ดี', 'ดีมาก', 'ถาด', 'ถาม', 'น่ารัก', 'น้ำ', 'น้ำจิ้ม', 'บริการ', 'บาท', 'ประทับใจ', 'พนักงาน', 'พอกับ', 'ภาพ', 'รสชาติ', 'ราคา', 'สรุป', 'สะอาดสะอ้าน', 'สัน', 'หลากหลาย', 'อร่อย', 'อันนี้', 'อาหาร', 'เคลม', 'เซ็ท', 'เต้าเจี้ยว', 'เนื้อ', 'เป็ด', 'เผ็ด', 'แบบนี้', 'แปลก', 'แฟน', 'โดยรวม', 'โมจิ', 'ใส่ใจ', 'ไม่ต้อง', 'ไอติม', '21', '30', '9', 'CTW', 'Terminal', 'กา', 'การงด', 'คิดถึง', 'ชาบู', 'ญี่ปุ่น', 'ฐาน', 'ทะยอย', 'ทำงาน', 'ที่ทาง', 'นาที', 'นึกถึง', 'พนักงานบริการ', 'พระราม', 'พรีเมียม', 'พาราไดซ์', 'มาตรา', 'มีโอ', 'รายละเอียด', 'ลูกค้า', 'สมาทาน', 'สาขา', 'สู้', 'ออร์เดอร์', 'เครื่องดื่ม', 'เค้า', 'เจอ', 'เป็นกันเอง', 'เหวี่ยง', 'แจ้ง', 'โม', 'โม่', 'โอเค', '', '++', '2', '555', '600', ':', 'กก', 'กรุบ', 'กั้น', 'ของหวาน', 'คนละ', 'คาว', 'คุ', 'คุ้มค่า', 'จาน', 'จิ้ม', 'จืด', 'ฉาก', 'ชอบ', 'ชาเขียว', 'ชิ้น', 'ซอย', 'ซอส', 'ตก', 'ตะ', 'ตัก', 'ตามน้ำ', 'ต่างกัน', 'ถั่ว', 'ถ

In [12]:
gensim_corpus = [dictionary.doc2bow(text, allow_update=True) for text in texts]

In [13]:
print(gensim_corpus)

[[(0, 1), (1, 1), (2, 1), (3, 5), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 2), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1), (19, 3), (20, 1), (21, 2), (22, 1), (23, 1), (24, 1), (25, 3), (26, 1), (27, 1), (28, 1), (29, 1), (30, 1), (31, 3), (32, 1), (33, 3), (34, 1), (35, 1), (36, 1), (37, 2), (38, 1), (39, 1), (40, 1), (41, 1), (42, 1), (43, 1), (44, 1), (45, 1), (46, 1), (47, 1)], [(7, 1), (12, 2), (16, 1), (19, 2), (22, 1), (33, 1), (48, 1), (49, 1), (50, 1), (51, 1), (52, 1), (53, 1), (54, 1), (55, 1), (56, 1), (57, 1), (58, 1), (59, 1), (60, 1), (61, 1), (62, 1), (63, 1), (64, 1), (65, 1), (66, 1), (67, 1), (68, 1), (69, 1), (70, 1), (71, 1), (72, 1), (73, 3), (74, 1), (75, 1), (76, 1), (77, 1), (78, 1), (79, 1), (80, 1), (81, 1), (82, 1), (83, 1), (84, 1)], [(3, 1), (8, 1), (10, 5), (11, 3), (17, 8), (18, 2), (22, 1), (25, 3), (26, 1), (27, 1), (31, 4), (37, 3), (38, 2), (47, 2), (56, 3), (57, 1), (85, 6), (86, 1), (87, 3), (88, 1), (89, 1),

In [14]:
word_frequencies = [[(dictionary[id], frequence) for id, frequence in couple] for couple in gensim_corpus]

In [15]:
print(word_frequencies)

[[('1', 1), ('4', 1), ('560', 1), ('กิน', 5), ('ข้างนอก', 1), ('ความต้องการ', 1), ('คอ', 1), ('คุณภาพ', 1), ('คุ้ม', 1), ('จำกัด', 1), ('ซุป', 1), ('ดำ', 1), ('ดี', 2), ('ดีมาก', 1), ('ถาด', 1), ('ถาม', 1), ('น่ารัก', 1), ('น้ำ', 1), ('น้ำจิ้ม', 1), ('บริการ', 3), ('บาท', 1), ('ประทับใจ', 2), ('พนักงาน', 1), ('พอกับ', 1), ('ภาพ', 1), ('รสชาติ', 3), ('ราคา', 1), ('สรุป', 1), ('สะอาดสะอ้าน', 1), ('สัน', 1), ('หลากหลาย', 1), ('อร่อย', 3), ('อันนี้', 1), ('อาหาร', 3), ('เคลม', 1), ('เซ็ท', 1), ('เต้าเจี้ยว', 1), ('เนื้อ', 2), ('เป็ด', 1), ('เผ็ด', 1), ('แบบนี้', 1), ('แปลก', 1), ('แฟน', 1), ('โดยรวม', 1), ('โมจิ', 1), ('ใส่ใจ', 1), ('ไม่ต้อง', 1), ('ไอติม', 1)], [('คุณภาพ', 1), ('ดี', 2), ('น่ารัก', 1), ('บริการ', 2), ('พนักงาน', 1), ('อาหาร', 1), ('21', 1), ('30', 1), ('9', 1), ('CTW', 1), ('Terminal', 1), ('กา', 1), ('การงด', 1), ('คิดถึง', 1), ('ชาบู', 1), ('ญี่ปุ่น', 1), ('ฐาน', 1), ('ทะยอย', 1), ('ทำงาน', 1), ('ที่ทาง', 1), ('นาที', 1), ('นึกถึง', 1), ('พนักงานบริการ', 1), ('พระราม', 

In [16]:
num_topics = 20
chunksize = 1000 # size of the doc looked at every pass
passes = 20 # number of passes through documents
iterations = 50
eval_every = 1  # Don't evaluate model perplexity, takes too much time.

temp = dictionary[0]  # This is only to "load" the dictionary.
id2word = dictionary.id2token

%time model = gensim.models.LdaModel(corpus=gensim_corpus, id2word=id2word, chunksize=chunksize, \
                       alpha='auto', eta='auto', \
                       iterations=iterations, num_topics=num_topics, \
                       passes=passes, eval_every=eval_every)

/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for

CPU times: user 719 ms, sys: 93.1 ms, total: 812 ms
Wall time: 741 ms


/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for

In [17]:
print(id2word)

{0: '1', 1: '4', 2: '560', 3: 'กิน', 4: 'ข้างนอก', 5: 'ความต้องการ', 6: 'คอ', 7: 'คุณภาพ', 8: 'คุ้ม', 9: 'จำกัด', 10: 'ซุป', 11: 'ดำ', 12: 'ดี', 13: 'ดีมาก', 14: 'ถาด', 15: 'ถาม', 16: 'น่ารัก', 17: 'น้ำ', 18: 'น้ำจิ้ม', 19: 'บริการ', 20: 'บาท', 21: 'ประทับใจ', 22: 'พนักงาน', 23: 'พอกับ', 24: 'ภาพ', 25: 'รสชาติ', 26: 'ราคา', 27: 'สรุป', 28: 'สะอาดสะอ้าน', 29: 'สัน', 30: 'หลากหลาย', 31: 'อร่อย', 32: 'อันนี้', 33: 'อาหาร', 34: 'เคลม', 35: 'เซ็ท', 36: 'เต้าเจี้ยว', 37: 'เนื้อ', 38: 'เป็ด', 39: 'เผ็ด', 40: 'แบบนี้', 41: 'แปลก', 42: 'แฟน', 43: 'โดยรวม', 44: 'โมจิ', 45: 'ใส่ใจ', 46: 'ไม่ต้อง', 47: 'ไอติม', 48: '21', 49: '30', 50: '9', 51: 'CTW', 52: 'Terminal', 53: 'กา', 54: 'การงด', 55: 'คิดถึง', 56: 'ชาบู', 57: 'ญี่ปุ่น', 58: 'ฐาน', 59: 'ทะยอย', 60: 'ทำงาน', 61: 'ที่ทาง', 62: 'นาที', 63: 'นึกถึง', 64: 'พนักงานบริการ', 65: 'พระราม', 66: 'พรีเมียม', 67: 'พาราไดซ์', 68: 'มาตรา', 69: 'มีโอ', 70: 'รายละเอียด', 71: 'ลูกค้า', 72: 'สมาทาน', 73: 'สาขา', 74: 'สู้', 75: 'ออร์เดอร์', 76: 'เครื่องดื่ม',

LDAvis
LDAvis is a web-based interactive visualisation of topics estimated using LDA. It provides a global view of the topics (and how they differ from each other), while at the same time allowing for a deep inspection of the terms most highly associated with each individual topic.

The package extracts information from a fitted LDA topic model to inform an interactive web-based visualization.

The visualisation has two basic pieces.

The left panel visualise the topics as circles in the two-dimensional plane whose centres are determined by computing the Jensen–Shannon divergence between topics, and then by using multidimensional scaling to project the inter-topic distances onto two dimensions. Each topic’s overall prevalence is encoded using the areas of the circles.

The right panel depicts a horizontal bar chart whose bars represent the individual terms that are the most useful for interpreting the currently selected topic on the left. A pair of overlaid bars represent both the corpus-wide frequency of a given term as well as the topic-specific frequency of the term.

The λ slider allows to rank the terms according to term relevance. By default, the terms of a topic are ranked in decreasing order according their topic-specific probability ( λ = 1 ). Moving the slider allows to adjust the rank of terms based on much discriminatory (or "relevant") are for the specific topic. The suggested “optimal” value of λ is 0.6.

In [18]:
import pyLDAvis.gensim as gensim
gensim.prepare(model, gensim_corpus, dictionary, mds='tsne')

ImportError: ignored

import pyLDAvis
import pyLDAvis.gensim
import pyLDAvis.sklearn
pyLDAvis.enable_notebook()

%matplotlib inline
data = gensimvis.prepare(model, gensim_corpus, dictionary)
pyLDAvis.enable_notebook()
pyLDAvis.display(data)

In [19]:
model.show_topic(18)

[('คน', 0.02451173),
 (':', 0.024455272),
 ('กิน', 0.019751256),
 ('รอ', 0.01854666),
 ('ทาน', 0.016504021),
 ('ดี', 0.016351976),
 ('ราคา', 0.015856642),
 ('บาท', 0.013883506),
 ('รสชาติ', 0.013692399),
 ('เดอะ', 0.01316155)]

In [20]:
df['topics'] = df['Review_tokenized'].apply(lambda x: model.get_document_topics(dictionary.doc2bow(x.split(',')))[0][0])
df['score'] = df['Review_tokenized'].apply(lambda x: model.get_document_topics(dictionary.doc2bow(x.split(',')))[0][1])

In [21]:
# print(df[['Restaurant', 'Review', 'topics', 'score']])
print(df[[ 'Review', 'topics']])

                                               Review  topics
0   ที่สำคัญของร้านนี้คือบริการดีมากพนักงานน่ารักส...       8
1   นึกถึงชาบูญี่ปุ่นยังไงก็ต้อง คิดถึงโมโม่ พาราไ...      14
2   มาทานช่วงนี้ สามารถนั่งโต๊ะเดียวกัน หม้อเดียวก...       6
3   ถ้านึกถึงชาบูที่มีเนื้อเน้นๆ ในราคาไม่โหดจนเกิ...       4
4   เดินมาหน้าร้านแล้วได้กลิ่นชาบูหอมมาก ๆ  ประกอบ...       8
5   ร้านบุฟเฟ่ ชาบูแนวญี่ปุ่น สายเนื้อหมู เนื้อวัว...      17
6   Number 20 : โมโม – พาราไดส์ (สาขาเดอะมอลบางกะป...       8
7   ร้านชาบูเฟรนไชส์รสชาติดีมากคุ้มค่าเหมาะสมกับรา...       8
8   มา านที่ขาบูชิต้องมาตอนหิว ไม่งั้นจะไม่คุ้มนะค...      10
9   ใครชอบกุ้งทอดเทมปุระ แค่กุ้งเทมปุระก็คุ้มแล้ว ...      12
10  กลับมาอัพเดทราคาชาบูชิ ตอนนี้อยู่ที่ 399 บาท n...       5
11  ห่างหายไปนานสำหรับชาบูชิ ตั้งแต่ทางร้านได้ปรับ...      11
12  เมื่อหลายวันก่อนนัดหาข้าวทานกับเดอะแกงค์ และก็...      18
13  บอกตรงๆว่าหลายครั้งที่เลือกจะกินบุฟเฟต์จะต้องอ...      18
14  สวัสดีครับวันนี้จะขอมารีวิวร้านชาบูชิ บุฟเฟขวั...       8
15  ชาบู

In [24]:
def topic2Array(x):
  topics = ""
  for i in range(len(x)):
    topics += x[i][0]
    print(topics)
    if i >= 9:
      break;
  return topics

In [25]:
array_topics = []
for i in range(model.num_topics):
  topic_list = topic2Array(model.show_topic(i))
  array_topics.append(topic_list)

print(len(array_topics))
print (array_topics)

ทาน
ทานดี
ทานดีชาบู
ทานดีชาบูราคา
ทานดีชาบูราคาเนื้อ
ทานดีชาบูราคาเนื้ออาหาร
ทานดีชาบูราคาเนื้ออาหารน้ำ
ทานดีชาบูราคาเนื้ออาหารน้ำอร่อย
ทานดีชาบูราคาเนื้ออาหารน้ำอร่อยสาขา
ทานดีชาบูราคาเนื้ออาหารน้ำอร่อยสาขาคุ้ม
ทาน
ทานดี
ทานดีชาบู
ทานดีชาบูราคา
ทานดีชาบูราคาเนื้อ
ทานดีชาบูราคาเนื้ออาหาร
ทานดีชาบูราคาเนื้ออาหารน้ำ
ทานดีชาบูราคาเนื้ออาหารน้ำอร่อย
ทานดีชาบูราคาเนื้ออาหารน้ำอร่อยสาขา
ทานดีชาบูราคาเนื้ออาหารน้ำอร่อยสาขาคุ้ม
ทาน
ทานดี
ทานดีชาบู
ทานดีชาบูราคา
ทานดีชาบูราคาเนื้อ
ทานดีชาบูราคาเนื้ออาหาร
ทานดีชาบูราคาเนื้ออาหารน้ำ
ทานดีชาบูราคาเนื้ออาหารน้ำอร่อย
ทานดีชาบูราคาเนื้ออาหารน้ำอร่อยสาขา
ทานดีชาบูราคาเนื้ออาหารน้ำอร่อยสาขาคุ้ม

ราคา
ราคาอาหาร
ราคาอาหารอร่อย
ราคาอาหารอร่อยสั่ง
ราคาอาหารอร่อยสั่งกิน
ราคาอาหารอร่อยสั่งกินซูชิ
ราคาอาหารอร่อยสั่งกินซูชิลอง
ราคาอาหารอร่อยสั่งกินซูชิลองจ่าย
ราคาอาหารอร่อยสั่งกินซูชิลองจ่ายบางส่วน
เนื้อ
เนื้อชอบ
เนื้อชอบรส
เนื้อชอบรสนึกถึง
เนื้อชอบรสนึกถึงเกินไป
เนื้อชอบรสนึกถึงเกินไปทาน
เนื้อชอบรสนึกถึงเกินไปทานชาบู
เนื้อชอบรสนึกถึงเกินไปทานชาบูราคา
เนื้อชอบ

In [26]:
for i in array_topics:
  print(i)

ทานดีชาบูราคาเนื้ออาหารน้ำอร่อยสาขาคุ้ม
ทานดีชาบูราคาเนื้ออาหารน้ำอร่อยสาขาคุ้ม
ทานดีชาบูราคาเนื้ออาหารน้ำอร่อยสาขาคุ้ม
ราคาอาหารอร่อยสั่งกินซูชิลองจ่ายบางส่วน
เนื้อชอบรสนึกถึงเกินไปทานชาบูราคาอร่อยสั่ง
ทานดีชาบูเนื้อน้ำเลือกซุปราคาสดบุฟเฟ่ต์
น้ำอร่อย:ซุปทานเลือกของหวานไก่ชาบู
บาทซุปราคาน้ำทานสาขารสชาติพิซซ่าคนละจ่าย
น้ำเลือกอร่อยกินชาบูทานเนื้อรสชาติซุปอาหาร
ทานดีชาบูราคาเนื้ออาหารน้ำอร่อยสาขาคุ้ม
อาหารคุ้มทานซูชิชิกุ้งราคามเทปุระ
กินสดกุ้งดีคนเวลาสำหรับเนื้ออาหารสาขา
ปุระมเทกุ้งทานชาบูทอดคุ้มชิซูชิ
ทานดีชาบูราคาเนื้ออาหารน้ำอร่อยสาขาคุ้ม
สาขาดีบริการชาบูอาหารพนักงานคุณภาพโอเคญี่ปุ่นโม
ทานดีชาบูราคาเนื้ออาหารน้ำอร่อยสาขาคุ้ม
เลือกหน้าหม้อทานอาหารชิสายพานห้าง3ค่า
ชอบดีชาบูเนื้ออร่อยคุ้มพนักงานเลือกของหวานสั่ง
คน:กินรอทานดีราคาบาทรสชาติเดอะ
ทานดีชาบูราคาเนื้ออาหารน้ำอร่อยสาขาคุ้ม
